SpaceX-Falcon 9 first stage landing prediction

# collecting the data:
In this capstone, we will predict if the falcon 9 first stage will land successfully. SpaceX advertises Falcon 9 rocket launches on its website with a cost of 69 million dollars. Other providers cost upward of 165 million dollars each, much of the savings is because spaceX can reuse first stage. There fore if we can determine if the first stage will land, we can determine the cost of the launch. thsi information can be used if an alternate company wants to bid against SpaceX for a rocket lauch. In this lab you will collect and make sure the data is in the correct format from an API. the following is an example of successful and launch

![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/lab_v2/images/landing\_1.gif)

several examples of unsuccessful landing are shown are:
![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/lab_v2/images/crash.gif)


most unsuccessful landings are planned. Space X performs a controlled landing in the oceans
### objectives:
In this lab we will make a get request to the Space X API. You will also do some basic data wrangling and formatting
<ul><li>Request to the spaceX api</li><li>clean the requested data</li>

## Import libraries and define auxiliary functions
we will import the following libraries into the lab

In [1]:
# Requests module is used to make http requests which we will use to get data from an API
import requests
import pandas as pd
import numpy as np
import datetime

In [2]:
'''this function is used to take the dataset and uses the rocket column to call the API and append the data to the list'''
def getBoosterVersion(data):
    for x in data['rocket']:
        response = requests.get('https://api.spacexdata.com/v4/rockets/'+str(x)).json()
        BoosterVersion.append(response['name'])

In [3]:
# we would like to know the name of the launch site being used,
# the longitude and latitude
'''this function takes the dataset and uses the launchpad column to call the API and append the data to the list'''
def getLaunchSite(data):
    for x in data['launchpad']:
        response = requests.get('https://api.spacexdata.com/v4/launchpads/'+str(x)).json()
        Longitude.append(response['longitude'])
        Latitude.append(response['latitude'])
        LaunchSite.append(response['name'])

In [4]:
# from the payload we would like to learn the mass of the payload and the orbit that it is going to:
'''Takes the dataset and uses the payloads column to call the API and append the data to the lists'''
def getPayloadData(data):
    for load in data['payloads']:
        response = requests.get('https://api.spacexdata.com/v4/payloads/'+load).json()
        PayloadMass.append(response['mass_kg'])
        Orbit.append(response['orbit'])

In [5]:
'''from cores we would like to learn the outcome of the landing, the type of landing, number of flights with that core,
Whether gridfins were used, whether the core is reused or legs or landing pad were reused'''
# takes the dataset and uses the cores column to call the API and append the data to the lists
def getCoreData(data):
    for core in data['cores']:
        if core['core'] != None:
            response = requests.get('https://api.spacexdata.com/v4/cores/'+core['core']).json()
            Block.append(response['block'])
            ReusedCount.append(response['reuse_count'])
            Serial.append(response['serial'])
        else:
            Block.append(None)
            ReusedCount.append(None)
            Serial.append(None)
        Outcome.append(str(core['landing_success'])+' '+str(core['landing_type']))
        Flights.append(core['flight'])
        GridFins.append(core['gridfins'])
        Reused.append(core['reused'])
        Legs.append(core['legs'])
        LandingPad.append(core['landpad'])

In [7]:
#let's request rocket launch data from spacex API with following url
spacex_url = 'https://api.spacexdata.com/v4/launches/past'
response = requests.get(spacex_url)

response.json()

### Data wrangling using an API

In [8]:
data = pd.json_normalize(response.json())
#data.head()

here we can notice that a lot of data are IDs. for example rocket column has no information about the rocket just an identification number
we will now use the API again to get information about launches using the IDs given for each launch.specifically we will be using columns rocket, payloads, launchpad and cores

data.columns

In [9]:
# let's take a subset of our dataframe keeping only the features we want and flight number and date_utc
data = data[['rocket','payloads','launchpad','cores','flight_number','date_utc']]

# we will remove rows with multiple cores because those are falcon rockets with 2 extra boosters and rows that have multiple payloads in a single rocket
data = data[data['cores'].map(len)==1]
data = data[data['payloads'].map(len)==1]


In [11]:
# since payloads and cores are list of size 1 we will also extract the single value in the list and replace the feature
data['cores'] = data['cores'].map(lambda x:x[0])
data['payloads'] = data['payloads'].map(lambda x:x[0])

In [12]:
# we also want to convert the datae_utc to a datetime datatype and then extracting the date leaving the time
data['date'] = pd.to_datetime(data['date_utc']).dt.date
# using the date we want we will restrict dates of the launches
date = data[data['date']<= datetime.date(2020, 11, 13)]

* from the <code>rocket</code> we would like to learn the booster name
* from the <code>payload</code> we would like to learn the mass of the payload and the orbit that it is going to
* from the <code>launchpad </code> we would like to know the name of the laucn sitebeing used, the longitude, and the latitude.
* from <code>cores</code> we would like to learn the outcome of the landing, the type of the landing, number of flights with that core, whether gridfins were used, whether the core is reused, whether legs were used, the landing pad used, the block of the core which is a number used to seperate version of cores, the number of times this specific core has been reused, and the serial of the core.

The data from these requests will be stored in lists and will be useed to create a new dataframe

In [13]:
#Global variables
BoosterVersion,PayloadMass,Orbit,LaunchSite = [],[],[],[]
Outcome,Flights,GridFins,Reused,Legs,LandingPad,Block = ([] for i in range(7))
ReusedCount,Serial,Longitude,Latitude = [],[],[],[]

 These functions will apply the outputs globally to the above lists. Take a look at <code>BoosterVersion</code> variable. Before we apply <code>getBoosterVersion</code> the list is empty

In [14]:
BoosterVersion

[]

In [15]:
# apply getBoosterVersion
getBoosterVersion(data)

In [16]:
# let's see data in BoosterVersion
BoosterVersion[0:6]

['Falcon 1', 'Falcon 1', 'Falcon 1', 'Falcon 1', 'Falcon 9', 'Falcon 9']

we can apply the rest of the function below:

In [17]:
# call getLaunchSite
getLaunchSite(data)

In [18]:
# call getPayloadData
getPayloadData(data)

In [19]:
# call getCoreData
getCoreData(data)

Finally lets construct our dataset using the data we have obtained. we can combine the columns into a dictionary.

In [20]:
launch_dict = {'FlightNumber':list(data['flight_number']),
              'Date':list(data['date']),
              'BoosterVersion':BoosterVersion,
              'PayloadMass':PayloadMass,
              'Orbit':Orbit,
              'LaunchSite':LaunchSite,
              'Outcome':Outcome,
              'Flights':Flights,
              'GridFins':GridFins,
              'Reused':Reused,
              'Legs':Legs,
              'LandingPad':LandingPad,
              'Block':Block,
              'ReusedCount':ReusedCount,
               'Serial':Serial,
               'Longitude':Longitude,
               'Latitude':Latitude
              }

In [21]:
# let's create a pandas dataframe by using launch_dict
data1 = pd.DataFrame(launch_dict)
data1.head()

,FlightNumber,Date,BoosterVersion,PayloadMass,Orbit,LaunchSite,Outcome,Flights,GridFins,Reused,Legs,LandingPad,Block,ReusedCount,Serial,Longitude,Latitude
0,1,2006-03-24,Falcon 1,20.0,LEO,Kwajalein Atoll,None None,1,False,False,False,None,NaN,0,Merlin1A,167.743129,9.047721
1,2,2007-03-21,Falcon 1,NaN,LEO,Kwajalein Atoll,None None,1,False,False,False,None,NaN,0,Merlin2A,167.743129,9.047721
2,4,2008-09-28,Falcon 1,165.0,LEO,Kwajalein Atoll,None None,1,False,False,False,None,NaN,0,Merlin2C,167.743129,9.047721
3,5,2009-07-13,Falcon 1,200.0,LEO,Kwajalein Atoll,None None,1,False,False,False,None,NaN,0,Merlin3C,167.743129,9.047721
4,6,2010-06-04,Falcon 9,NaN,LEO,CCSFS SLC 40,None None,1,False,False,False,None,1.0,0,B0003,-80.577366,28.561857


Filter the dataframe to include only 'Falcon 9' launches

In [23]:
data_falcon9 = data1[data1['BoosterVersion']=='Falcon 9']
data_falcon9.head()

,FlightNumber,Date,BoosterVersion,PayloadMass,Orbit,LaunchSite,Outcome,Flights,GridFins,Reused,Legs,LandingPad,Block,ReusedCount,Serial,Longitude,Latitude
4,6,2010-06-04,Falcon 9,NaN,LEO,CCSFS SLC 40,None None,1,False,False,False,None,1.0,0,B0003,-80.577366,28.561857
5,8,2012-05-22,Falcon 9,525.0,LEO,CCSFS SLC 40,None None,1,False,False,False,None,1.0,0,B0005,-80.577366,28.561857
6,10,2013-03-01,Falcon 9,677.0,ISS,CCSFS SLC 40,None None,1,False,False,False,None,1.0,0,B0007,-80.577366,28.561857
7,11,2013-09-29,Falcon 9,500.0,PO,VAFB SLC 4E,False Ocean,1,False,False,False,None,1.0,0,B1003,-120.610829,34.632093
8,12,2013-12-03,Falcon 9,3170.0,GTO,CCSFS SLC 40,None None,1,False,False,False,None,1.0,0,B1004,-80.577366,28.561857


we ahve removed some records so we need to reset the FlightNumber

In [24]:
data_falcon9.loc[:,'FlightNumber'] = list(range(1, data_falcon9.shape[0]+1))
data_falcon9.head()

C:\Users\THIRUMALESHWARAREDDY\anaconda3\lib\site-packages\pandas\core\indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


,FlightNumber,Date,BoosterVersion,PayloadMass,Orbit,LaunchSite,Outcome,Flights,GridFins,Reused,Legs,LandingPad,Block,ReusedCount,Serial,Longitude,Latitude
4,1,2010-06-04,Falcon 9,NaN,LEO,CCSFS SLC 40,None None,1,False,False,False,None,1.0,0,B0003,-80.577366,28.561857
5,2,2012-05-22,Falcon 9,525.0,LEO,CCSFS SLC 40,None None,1,False,False,False,None,1.0,0,B0005,-80.577366,28.561857
6,3,2013-03-01,Falcon 9,677.0,ISS,CCSFS SLC 40,None None,1,False,False,False,None,1.0,0,B0007,-80.577366,28.561857
7,4,2013-09-29,Falcon 9,500.0,PO,VAFB SLC 4E,False Ocean,1,False,False,False,None,1.0,0,B1003,-120.610829,34.632093
8,5,2013-12-03,Falcon 9,3170.0,GTO,CCSFS SLC 40,None None,1,False,False,False,None,1.0,0,B1004,-80.577366,28.561857


### Data wrangling:

In [25]:
# let us is there any null values present in the dataframe
data_falcon9.isnull().sum()

FlightNumber       0
Date               0
BoosterVersion     0
PayloadMass       22
Orbit              1
LaunchSite         0
Outcome            0
Flights            0
GridFins           0
Reused             0
Legs               0
LandingPad        26
Block              0
ReusedCount        0
Serial             0
Longitude          0
Latitude           0
dtype: int64

### Dealing the missing values:

In [26]:
# impute missing values in Payloadmass with mean()
data_falcon9['PayloadMass'].fillna(data_falcon9['PayloadMass'].mean(),inplace=True)
data_falcon9.isnull().sum()

C:\Users\THIRUMALESHWARAREDDY\anaconda3\lib\site-packages\pandas\core\generic.py:6392: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)


FlightNumber       0
Date               0
BoosterVersion     0
PayloadMass        0
Orbit              1
LaunchSite         0
Outcome            0
Flights            0
GridFins           0
Reused             0
Legs               0
LandingPad        26
Block              0
ReusedCount        0
Serial             0
Longitude          0
Latitude           0
dtype: int64